# Steps to Automate CRM Reporting



Import Pandas and Numpy Library

In [ ]:
import pandas as pd
import numpy as np



## Import all the required files 
4 input files - CRM file - Installation File - Master file. Parse the dates in all the files and take only the required columns

In [ ]:
inst_1 = pd.read_excel('Mar Installation.xlsx', usecols='AK,A,E,BB,K,BF,BG,AS', parse_dates=['JOBCREATIONDATE'])

inst_2 = pd.read_excel('Mar Installation.xlsx', usecols='AL,A,E,BB,K,BF,BG,AS', parse_dates=['JOBCREATIONDATE'])

crm = pd.read_excel('CRM Data from 25th Feb - 9th Mar.xlsx', usecols='B,M:N,T:U')

fh_web = pd.read_excel('FH Website.xls', parse_dates=['Date'], usecols='A:D,G,I')

fw_web = pd.read_excel('FW Website.xls', parse_dates=['Date'], usecols='B:E,H,J')

chatbot = pd.read_excel('chatbot (2).xls', parse_dates=['Create date'], usecols='A:C,F:G,I')

ask_expert = pd.read_excel('ask_experts (3).xls', parse_dates=['Date'], usecols='A,C')

master = pd.read_excel("Mar'18 Digital Leads Master.xlsx", parse_dates=['Date Registered'])

### Cleaning the Master file 
1. Sort the dataset by date in asceding order
2. Remove duplicate rows by Mobile number

In [ ]:
# Find the total number of rows and columns
master.shape

In [ ]:
# Get Column names of the master dataset
master.columns

In [ ]:
del master['Sr no']

In [ ]:
master['Purchase Date'] = pd.to_datetime(master['Purchase Date'])

In [ ]:
# Sorting the dataset by date and keeping only the first value of the mobile no
master_deduped = master.sort_values('Date Registered').drop_duplicates('Mobile no')

In [ ]:
master_deduped.head(3)

### Setting up ask an expert dataset
1. Add Column - Source
2. Rename Columns

In [ ]:
ask_expert['source'] = 'Ask a Expert'
ask_expert.columns = ['date', 'mobile', 'source']

### Setting up Chatbot dataset
1. Add Column - Source
2. Rename columns

In [ ]:
chatbot['source'] = 'Chatbot'
chatbot.columns = ['name', 'email', 'mobile', 'date','utm_source', 'utm_campaign', 'source']


### Setting up FH Website dataset
1. Add Column - Source
2. Rename columns

In [ ]:
fh_web['source'] = 'FH Website'
fh_web.columns = ['date', 'name', 'email', 'mobile', 'utm_source', 'utm_campaign', 'source']

### Setting up FW Website dataset
1. Add Column - Source
2. Rename columns

In [ ]:
fw_web['source'] = 'FW Website'
fw_web.columns = ['date', 'name', 'email', 'mobile', 'utm_source', 'utm_campaign', 'source']

## Concatenating and Cleaning the 4 datasets
1. Concantenate the 4 dataframes
2. Sort by mobile desc
3. Remove duplicates based on mobile no
4. Sort by date and source asc

In [ ]:
df_list = [ask_expert, chatbot, fw_web, fh_web]

In [ ]:
df_concat = pd.concat(df_list, ignore_index=True)

In [ ]:
len(ask_expert) + len(chatbot) + len(fw_web) + len(fh_web) == len(df_concat)

In [ ]:
df_concat_depuded= df_concat.sort_values(by='mobile', ascending=[False])\
                            .drop_duplicates(subset='mobile')\
                            .sort_values(by=['date', 'source'], ascending= [True,True])

In [ ]:
df_concat_depuded.head(3)

In [ ]:
crm['Lead Status'].value_counts()

In [ ]:
crm[crm['Mobile:'] == 9716299795]

In [ ]:
crm['lead_rank'] = crm['Lead Status'].apply(lead_ranking)

In [ ]:
crm[crm['Lead Status'] == 'Demo Booked'].sort_values('Mobile:').head(3)

In [ ]:
crm['Activity/Campaign'] = crm['Activity/Campaign'].astype('object')

In [ ]:
act = crm['Activity/Campaign']

In [ ]:
act = act.fillna(value="")

## Setting up the CRM dataset

1. Create function to add column which ranks based on lead status
2. Create function to add column which ranks based on activity/ campaign
3. Fill the NaN values in activity column with blank values
3. Rename all columns
4. Sort by mobile, lead_rank, campaign rank in asc order
5. Remove duplicates based on mobile, keep first value only

In [ ]:
def lead_ranking(s):
    if s == 'Demo Booked':
        return 1
    elif s == 'Open':
        return 2
    elif s == 'New':
        return 3
    elif s == 'Closed Lost':
        return 4
    else:
        return 5

In [ ]:
def activity_campaign(s):
    if 'cpc' in s or 'cpl' in s:
        return 1
    else:
        return 2

In [ ]:
crm.columns = ['ass_user', 'mobile', 'city', 'status', 'campaign']

In [ ]:
crm['campaign'] = crm['campaign'].fillna(value="")

In [ ]:
crm['lead_rank'] = crm['status'].apply(lead_ranking)

In [ ]:
crm['campaign_rank'] = crm['campaign'].str.lower().apply(activity_campaign)

In [ ]:
crm_deduped = crm.sort_values(by=['mobile', 'lead_rank', 'campaign_rank'], ascending=[False, True, True])\
                .drop_duplicates('mobile')

In [ ]:
crm_deduped['mobile'].head(3)

In [ ]:
inst_1.head(3)

In [ ]:
inst_2.head(3)

In [ ]:
inst_1 = inst_1.dropna(how='all')

In [ ]:
inst_2 = inst_2.dropna(how = 'all')

In [ ]:
inst_2 = inst_2.dropna(how='any', subset=['CUSTOMER_PHONE_NO'])

In [ ]:
inst_1.columns = ['status', 'jobsheetno', 'model', 'mobile', 'date_time', 'date_of_purchase', 'dealer', 'distributor']
inst_2.columns = ['status', 'jobsheetno', 'model', 'mobile', 'date_time', 'date_of_purchase', 'dealer', 'distributor']

In [ ]:
inst_concat = pd.concat([inst_1,inst_2], ignore_index=True)

In [ ]:
inst_concat['mobile'].astype('int64')